In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
'''
numpy is going to complain about not being able to determine a dtype
when aggregating, but it can be safely ignored
'''
np.warnings.filterwarnings('ignore')

In [3]:
loc = './dryden_alex_ODM_final/raw_data/DOF_Valuation/tc234.csv'
valuation_df = pd.read_csv(loc)


/Users/alexdryden/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,4,45,46,53,76,77,90,97,100,103,106,109) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Complaints and the Problems that issue fromt them are stored in two seperate datafiles. The foriegn key is a complaint id. So we will set the index on both and join them. 

,BBLE,BORO,BLOCK,LOT,EASE,SECVOL,DISTRICT,YEAR4,CUR_FV_L,CUR_FV_T,...,O_AT_GRP,O_APPLIC,REUC,GEO_RC,COOP_NUM,EX_INDS,EX_COUNT,EX_CHGDT,DCHGDT,SM_CHGDT
0,1000010010,1,1,10,NaN,101,51,2009,192000000,282200000,...,0,NaN,NaN,0,0,EEEE,1,00/00/0000,11/07/2003,10/27/2009
1,1000010101,1,1,101,NaN,101,85,2009,8940000,20400000,...,0,NaN,NaN,0,0,EEEE,1,00/00/0000,02/03/1994,01/01/2010
2,1000010201,1,1,201,NaN,101,51,2009,27600000,186150000,...,0,NaN,NaN,0,0,EEEE,1,00/00/0000,12/17/1996,09/21/2009
3,1000020001,1,2,1,NaN,101,51,2009,77400000,95300000,...,0,NaN,NaN,12,0,EEEE,1,00/00/0000,08/27/2001,01/01/2010
4,1000020023,1,2,23,NaN,101,85,2009,32300000,37300000,...,0,NaN,NaN,12,0,EEEE,1,00/00/0000,01/17/2007,01/01/2010


In [7]:

#add leading zeros to the bloack and lot so they concat with boro into standard 10 digit
valuation_df['BLOCK'] = valuation_df['BLOCK'].apply(lambda x: '{0:0>5}'.format(x))
valuation_df['LOT'] = valuation_df['LOT'].apply(lambda x: '{0:0>4}'.format(x))

#make BBL column
valuation_df['BBL'] = valuation_df['BORO'].map(str) + valuation_df['BLOCK'].map(str) + valuation_df['LOT'].map(str)



In [9]:
valuation_df.set_index('BBL', inplace=True)

In [33]:
f1 = valuation_df[['OWNER', 'TOT_UNIT', 'RES_UNIT','BLDGS', 'STORY', 'CUR_FV_L', 'CUR_FV_T', 'CURAVL', 'CURAVT', 'CURAVL_A','CURAVT_A','EX_INDS', 'FN_EXT_A']]

"We phase in changes to the assessed value of Class 2 properties with more than 10 units and Class 4 properties over a five-year period. This means that we apply 20% of the change each year for five years.

"In any given year, there are multiple transitions being applied which results in an actual assessed value and a transitional assessed value for your property each year.  The law requires that we use whichever number is lower – your actual assessed value or transitional assessed value – to get to your property tax bill."
- NYC DoF (https://www1.nyc.gov/site/finance/taxes/property-determining-your-transitional-assessed-value.page)

In [34]:
f1.columns=[
    'Owner', 
    'Total_Units', 
    'Residential_Units', 
    'Buildings', 
    'Stories', 
    'Land_Market_Value', 
    'Total_Market_Value',
    'Transitional_Assesed_Land_Value',
    'Transitional_Assessed_Total_Value,',
    'Actual_Assesed_Land_Value',
    'Actual_Assessed_Total_Value,',
    '5_Year_Exemption_Flags',
    'Exempt_Total_Value'
]

In [35]:
#save file
f1.to_csv(r'./dryden_alex_ODM_final/intermediate_data/intermediate_csv/DoFValuationClean.csv', index='BBL', header=True)

In [36]:
f1.head()

,Owner,Total_Units,Residential_Units,Buildings,Stories,Land_Market_Value,Total_Market_Value,Transitional_Assesed_Land_Value,"Transitional_Assessed_Total_Value,",Actual_Assesed_Land_Value,"Actual_Assessed_Total_Value,",5_Year_Exemption_Flags,Exempt_Total_Value
BBL,,,,,,,,,,,,,
1000010010,UNITED STATES OF AMER,0,0,1,0.0,192000000,282200000,74997000,113373000,86400000,126990000,EEEE,136818900
1000010101,U S GOVT LAND & BLDGS,0,0,1,0.0,8940000,20400000,3540600,8082000,4023000,9180000,EEEE,9630000
1000010201,U S GOVT LAND & BLDGS,0,0,1,0.0,27600000,186150000,9959400,77828400,12420000,83767500,EEEE,87210000
1000020001,DEPT OF GENERAL SERVI,0,0,1,3.0,77400000,95300000,29704050,37372770,34830000,42885000,EEEE,47108700
1000020023,DEPARTMENT OF BUSINES,0,0,1,2.0,32300000,37300000,12798000,14769000,14535000,16785000,EEEE,17640000
